In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2

In [3]:
model = load_model('models/facialPointsPredictor.h5')

def get_points(image):
    image = tf.reshape(image, (96, 96, 1))
    points = model.predict(np.array([image]))[0]

    points2 = []
    for i, co in enumerate(points[0::2]):
        points2.append((co, points[1::2][i]))

    ancho_gafas= int((points2[7][0]-points2[9][0]))
    alto_gafas = int((points2[10][1]-points2[8][1]))

    print(ancho_gafas, alto_gafas)

    return points[0::2], points[1::2]

In [6]:
cap = cv2.VideoCapture(0)
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
glassesCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lowerbody.xml')

if not cap.isOpened():
    print("Cannot open camera")
    exit()

cont = 0
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here

    frame = cv2.flip(frame, 90)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
    gafas = cv2.imread('filtros/sunglasses.png', cv2.IMREAD_UNCHANGED)

    for (x, y, w, h) in faces:
        gray_face = frame[y:y+h, x:x+w]
        original_shape=gray_face.shape
        frame_cara = cv2.resize(frame[y:y + h, x:x + w], (96, 96)) / 255

        points_x, points_y = get_points(frame_cara)

        points = list(zip(points_x, points_y))

        ancho_gafas= int((points[7][0]-points[9][0]))
        alto_gafas = int((points[10][1]-points[8][1]))

        print(ancho_gafas, alto_gafas)

        if ancho_gafas > 0 and alto_gafas > 0:
            gafas_resized = cv2.resize(gafas, (ancho_gafas, alto_gafas), interpolation = cv2.INTER_CUBIC)

            region_no_transparente = gafas_resized[:,:,:3] != 0
            frame_cara[int(points[9][1]):int(points[9][1])+alto_gafas,
            int(points[9][0]):int(points[9][0])+ancho_gafas,:][region_no_transparente] = gafas_resized[:,:,:3][region_no_transparente]

            frame[y:y+h, x:x+w] = cv2.resize(frame_cara,  (96, 96), interpolation = cv2.INTER_CUBIC)

        for point in points:
            #resize points coordinates to original dataframe
            point = (point[0] * w / 96, point[1] * h / 96)

            #draw cirlce
            cv2.circle(frame, (int(point[0] + x), int(point[1] + y)), 1, (255, 255, 255), 3)

    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

    cont += 1
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step
-42 25
-42 25
1/1 [==============================] - 0s 22ms/step
-62 28
-62 28
1/1 [==============================] - 0s 22ms/step
-64 29
-64 29
1/1 [==============================] - 0s 22ms/step
-61 28
-61 28
1/1 [==============================] - 0s 22ms/step
-61 23
-61 23
1/1 [==============================] - 0s 22ms/step
-9 25
-9 25
1/1 [==============================] - 0s 23ms/step
-53 24
-53 24
1/1 [==============================] - 0s 23ms/step
-57 22
-57 22
1/1 [==============================] - 0s 21ms/step
-57 23
-57 23
1/1 [==============================] - 0s 22ms/step
-55 23
-55 23
1/1 [==============================] - 0s 22ms/step
-61 23
-61 23
1/1 [==============================] - 0s 23ms/step
3 25
3 25


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed